In [1]:
!pip install openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.5/76.5 kB 1.5 MB/s eta 0:00:00


In [2]:
import getpass, os
secret_key = getpass.getpass('Enter OpenAI secret key:')
os.environ['OPENAI_API_KEY'] = secret_key

Enter OpenAI secret key:··········


In [3]:
import openai

## New Spanish alignment


- [ ] Fix the output parsing part of the notebook. Perhaps make a function called `get_alignments_from_prompt_output()` instead of just splitting on highly specific string values
- [ ] (TODO LATER) Programmatically load a Spanish translation from eBible corpus


Some ideas for improving the initial prompt.

- [x] Add language typology information about Spanish
- [x] Add information about translation style of specific Spanish translation we are using (more literal vs. more dynamic)
- [x] Add more examples?
- [x] Replace current examples with examples from the language (e.g., Spanish)
- [x] Move the rationale of each alignment before the alignment itself
- [x] Experiment with tweaking the system prompt to see if it helps
- [x] SIMPLIFY the prompt down to just the most relevant content (do we need a rationale? Does it work without? We can probably drop out relevant grammatical patterns)

In [4]:
prompt = '''
Here are some general facts to note about Spanish:
Spanish is a fusional language, ensure correct affix attachment; follow SVO order; mark verbs for tense, aspect, mood.
For translating from Greek: replace Greek's three-gender system with Spanish's two-gender system, ensuring agreement; shift to SVO order; adapt Greek Voice/Aspect/Mood markings to Spanish system.

Translation style:
The Spanish translation is a balanced approach between a literal translation and a more dynamic translation. This means that there may be slight differences in phrase order and other adaptations to produce a more natural translation.

Here is a sentence:
English: And He said to them What things; - And they said to Him The things concerning Jesus of Nazareth, who was a man a prophet mighty in deed and word before - God and all the people,
Greek: καὶ εἶπεν αὐτοῖς Ποῖα;οἱ δὲ εἶπαν αὐτῷ Τὰ περὶ Ἰησοῦ τοῦ Ναζαρηνοῦ,ὃς ἐγένετο ἀνὴρ προφήτης δυνατὸς ἐν ἔργῳ καὶ λόγῳ ἐναντίον τοῦ Θεοῦ καὶ παντὸς τοῦ λαοῦ,
Spanish: —¿Qué es lo que ha pasado? —preguntó. Ellos respondieron: —Lo de Jesús de Nazaret. Era un profeta poderoso en obras y en palabras delante de Dios y de todo el pueblo.

Here is a phonological, semantic, orthographic alignment of that sentence:
```
[
    {
        "Spanish phrase": "—preguntó.",
        "Target phrase": "And He said to them",
        "Greek phrase": "καὶ εἶπεν αὐτοῖς"
    },
    {
        "Spanish phrase": "¿Qué es lo que ha pasado?",
        "Target phrase": "What things;",
        "Greek phrase": "Ποῖα;"
    },
    {
        "Spanish phrase": "Ellos respondieron",
        "Target phrase": "And they said to Him",
        "Greek phrase": "οἱ δὲ εἶπαν αὐτοῖς"
    },
    {
        "Spanish phrase": "—Lo de Jesús de Nazaret.",
        "Target phrase": "The things concerning Jesus of Nazareth,",
        "Greek phrase": "Τὰ περὶ Ἰησοῦ τοῦ Ναζαρηνοῦ"
    },
    {
        "Spanish phrase": "Era un profeta poderoso en obras y en palabras",
        "Target phrase": "who was a man a prophet mighty in deed and word",
        "Greek phrase": "ὃς ἐγένετο ἀνὴρ προφήτης δυνατὸς ἐν ἔργῳ καὶ λόγῳ"
    },
    {
        "Spanish phrase": "delante de Dios y de todo el pueblo.",
        "Target phrase": "before - God and all the people",
        "Greek phrase": "ἐναντίον τοῦ Θεοῦ καὶ παντὸς τοῦ λαοῦ"
    }
]

```

Please also align the following sentence. Avoid including multiple phrases in a single alignment unit. You may need to break phrases  on commas or other major punctuation, including enclosing quotation marks. But you may also need to break a phrase along conjunctions or other words that typically mark the start of a new phrase:

English: After  now  these things  appointed  the  Lord  others  seventy, and  sent  them  in  two [by]  before  [the] face  of Himself  into  every  city  and  place  where  was about  He Himself  to go.
Greek: Μετὰ δὲ ταῦτα ἀνέδειξεν ὁ Κύριος ἑτέρους ἑβδομήκοντα,καὶ ἀπέστειλεν αὐτοὺς ἀνὰ δύο πρὸ προσώπου αὐτοῦ εἰς πᾶσαν πόλιν καὶ τόπον οὗ ἤμελλεν αὐτὸς ἔρχεσθαι.
Spanish: Después de esto, el Señor designó a otros setenta, y los envió de dos en dos delante de Él, a toda ciudad y lugar adonde Él había de ir.
'''

messages = [
    {"role": "system", "content": f"You are LangAlignerGPT. Analyze the user-supplied alignment examples below and follow any instructions the user gives."},
    {"role": "user", "content": prompt},
]

response = openai.ChatCompletion.create(
    model="gpt-4",
    messages=messages,
    temperature=0.3,
    n=1,
    presence_penalty=0.5,
    frequency_penalty=0.5,
)

generated_texts = [
    choice.message["content"].strip() for choice in response["choices"]
]
print(generated_texts[0])

Here is a phonological, semantic, orthographic alignment of the sentence:

```
[
    {
        "Spanish phrase": "Después de esto",
        "Target phrase": "After now these things",
        "Greek phrase": "Μετὰ δὲ ταῦτα"
    },
    {
        "Spanish phrase": ", el Señor designó a otros setenta",
        "Target phrase": "appointed the Lord others seventy",
        "Greek phrase": "ἀνέδειξεν ὁ Κύριος ἑτέρους ἑβδομήκοντα"
    },
    {
        "Spanish phrase": ", y los envió de dos en dos delante de Él",
        "Target phrase": ", and sent them in two [by] before [the] face of Himself",
        "Greek phrase": ",καὶ ἀπέστειλεν αὐτοὺς ἀνὰ δύο πρὸ προσώπου αὐτοῦ"
    },
    {
        "Spanish phrase": ", a toda ciudad y lugar adonde Él había de ir.",
        "Target phrase": ", into every city and place where was about He Himself to go.",
        "Greek phrase": 	"εἰς πᾶσαν πόλιν καὶ τόπον οὗ ἤμελλεν αὐτὸς ἔρχεσθαι."
    }
]
```


## Align individual chunks from output

In [5]:
import json

def get_alignments_from_prompt_output(generated_texts):
  start_index = generated_texts.find('```')
  end_index = generated_texts.rfind('```')
  json_data = generated_texts[start_index + 3:end_index]
  json_data = json_data.strip()
  data = json.loads(json_data)
  # json.dumps(data, indent=4, ensure_ascii=False)
  return data


In [6]:
output = get_alignments_from_prompt_output(generated_texts[0])

In [7]:
output

[{'Spanish phrase': 'Después de esto',
  'Target phrase': 'After now these things',
  'Greek phrase': 'Μετὰ δὲ ταῦτα'},
 {'Spanish phrase': ', el Señor designó a otros setenta',
  'Target phrase': 'appointed the Lord others seventy',
  'Greek phrase': 'ἀνέδειξεν ὁ Κύριος ἑτέρους ἑβδομήκοντα'},
 {'Spanish phrase': ', y los envió de dos en dos delante de Él',
  'Target phrase': ', and sent them in two [by] before [the] face of Himself',
  'Greek phrase': ',καὶ ἀπέστειλεν αὐτοὺς ἀνὰ δύο πρὸ προσώπου αὐτοῦ'},
 {'Spanish phrase': ', a toda ciudad y lugar adonde Él había de ir.',
  'Target phrase': ', into every city and place where was about He Himself to go.',
  'Greek phrase': 'εἰς πᾶσαν πόλιν καὶ τόπον οὗ ἤμελλεν αὐτὸς ἔρχεσθαι.'}]

In [8]:
final_alignments = []

for chunk in output:

    prompt = '''Here is a phrase:
      {POS_aligned_chunks}

      Please further align and break down the provided chunk into a mapping of the fewest possible tokens (sometimes multiple tokens will align to one token; that's expected) in a format similar to the following:

      E.g.: "Rationale": "Semantic alignment; Orthographic (comma)","Source phrase": "Después de esto,",\n\t"Spanish phrase": "After now these things",\n\t"Greek phrase": "Μετὰ δὲ ταῦτα"

      - Source token(s)\t\t-->\tTarget token(s)\t\t-->\tGreek token(s)
      - Después de\t\t-->\tAfter now\t\t-->\tΜετὰ δὲ
      - esto\t\t-->\tthese things\t\t-->\tταῦτα

      Do not include any other information or commentary. Only tell me what the alignment is.

      The chunk I want you to align is:
        ```
        {current_chunk}
        ```
      '''.format(POS_aligned_chunks=json.dumps(output, ensure_ascii = False), current_chunk=json.dumps(chunk, ensure_ascii = False))

    messages = [
        {"role": "system", "content": f"You are LangAlignerGPT. Analyze the user-supplied alignment examples below and follow any instructions the user gives."},
        {"role": "user", "content": prompt},
    ]

    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=messages,
        temperature=0.3,
        n=1,
        presence_penalty=0.5,
        frequency_penalty=0.5,
    )

    generated_texts_for_chunk = [
        choice.message["content"].strip() for choice in response["choices"]
    ]
    print(generated_texts_for_chunk[0])
    final_alignments.append(generated_texts_for_chunk[0])

- Source token(s)		-->	Target token(s)		-->	Greek token(s)
- Después de		-->	After now		-->	Μετὰ δὲ
- esto		        -->	these things	        -->	ταῦτα
- Source token(s)		-->	Target token(s)		-->	Greek token(s)
- , el Señor		-->	the Lord		-->	ὁ Κύριος
- designó a 	        -->	appointed	        -->	ἀνέδειξεν
- otros setenta	        -->	others seventy	        -->	ἑτέρους ἑβδομήκοντα
- Source token(s)		-->	Target token(s)		-->	Greek token(s)
- , y los envió		-->	, and sent them		-->	,καὶ ἀπέστειλεν αὐτοὺς
- de dos en dos 	        -->	in two [by] 	                -->	ἀνὰ δύο
- delante de Él		        -->	before [the] face of Himself  -->  πρὸ προσώπου αὐτοῦ
- Source token(s)		-->	Target token(s)		-->	Greek token(s)
- , a toda ciudad y lugar  -->  , into every city and place  --> εἰς πᾶσαν πόλιν καὶ τόπον
- adonde Él había de ir.   --> where was about He Himself to go.   --> οὗ ἤμελλεν αὐτὸς ἔρχεσθαι.


In [9]:
for i in final_alignments:
  print(i)
  print('-----')

- Source token(s)		-->	Target token(s)		-->	Greek token(s)
- Después de		-->	After now		-->	Μετὰ δὲ
- esto		        -->	these things	        -->	ταῦτα
-----
- Source token(s)		-->	Target token(s)		-->	Greek token(s)
- , el Señor		-->	the Lord		-->	ὁ Κύριος
- designó a 	        -->	appointed	        -->	ἀνέδειξεν
- otros setenta	        -->	others seventy	        -->	ἑτέρους ἑβδομήκοντα
-----
- Source token(s)		-->	Target token(s)		-->	Greek token(s)
- , y los envió		-->	, and sent them		-->	,καὶ ἀπέστειλεν αὐτοὺς
- de dos en dos 	        -->	in two [by] 	                -->	ἀνὰ δύο
- delante de Él		        -->	before [the] face of Himself  -->  πρὸ προσώπου αὐτοῦ
-----
- Source token(s)		-->	Target token(s)		-->	Greek token(s)
- , a toda ciudad y lugar  -->  , into every city and place  --> εἰς πᾶσαν πόλιν καὶ τόπον
- adonde Él había de ir.   --> where was about He Himself to go.   --> οὗ ἤμελλεν αὐτὸς ἔρχεσθαι.
-----
